## Import

In [ ]:
import random
import pandas as pd
import numpy as np
import os
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import sklearn

import timm
from torchvision.models import video
import torchvision

import warnings
warnings.filterwarnings(action='ignore') 



In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## Hyperparameter Setting

In [ ]:
class CFG:
    model_name= "r3d_18"
    n_folds = 5
    n_classes = 13
    video_length=50
    img_size=128
    epochs=50
    lr=3e-4
    batch_size=16
    seed=41
    earlystop=10
        

## Fixed RandomSeed

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG.seed) # Seed 고정

## Data Load

In [ ]:
df = pd.read_csv('./train_detail_classified.csv')
df.head(30)

## 항목 별로 split -> 후에 stratified k fold 전략 사용

In [ ]:
#-1 라벨인 데이터 버리기
#df_crush=df[df['crush']!=1]
df_ego=df[df['ego']!=-1]
df_weather=df[df['weather']!=-1]
df_timing=df[df['timing']!=-1]

In [ ]:
train_crush, val_crush, _, _ = train_test_split(df, df['crush'], test_size=0.2, random_state=CFG.seed)
train_ego, val_ego, _,_=train_test_split(df_ego, df_ego['ego'], test_size=0.2, random_state=CFG.seed)
train_weather, val_weather, _,_=train_test_split(df_weather, df_weather['weather'], test_size=0.2, random_state=CFG.seed)
train_timing, val_timing,_,_=train_test_split(df_timing, df_timing['timing'], test_size=0.2, random_state=CFG.seed)

In [ ]:
print(len(train_timing), len(val_timing))

## CustomDataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, video_path_list, label_list):
        self.video_path_list = video_path_list
        self.label_list = label_list
        
    def __getitem__(self, index):
        frames = self.get_video(self.video_path_list[index])
        
        if self.label_list is not None:
            label = self.label_list[index]
            return frames, label
        else:
            return frames
        
    def __len__(self):
        return len(self.video_path_list)
    
    def get_video(self, path):
        frames = []
        cap = cv2.VideoCapture(path)
        for _ in range(CFG.video_length):
            _, img = cap.read()
            img = cv2.resize(img, (CFG.img_size, CFG.img_size))
            img = img / 255.
            frames.append(img)
        return torch.FloatTensor(np.array(frames)).permute(3, 0, 1, 2)

In [ ]:
train_crush_dataset = CustomDataset(train_crush['video_path'].values, train_crush['crush'].values)
train_crush_loader = DataLoader(train_crush_dataset, batch_size = CFG.batch_size, shuffle=True, num_workers=2)

val_crush_dataset = CustomDataset(val_crush['video_path'].values, val_crush['crush'].values)
val_crush_loader = DataLoader(val_crush_dataset, batch_size = CFG.batch_size, shuffle=False, num_workers=2)

In [ ]:
train_ego_dataset = CustomDataset(train_ego['video_path'].values, train_ego['ego'].values)
train_ego_loader = DataLoader(train_ego_dataset, batch_size = CFG.batch_size, shuffle=True, num_workers=2)

val_ego_dataset = CustomDataset(val_ego['video_path'].values, val_ego['ego'].values)
val_ego_loader = DataLoader(val_ego_dataset, batch_size = CFG.batch_size, shuffle=False, num_workers=2)

In [ ]:
train_weather_dataset = CustomDataset(train_weather['video_path'].values, train_weather['weather'].values)
train_weather_loader = DataLoader(train_weather_dataset, batch_size = CFG.batch_size, shuffle=True, num_workers=2)

val_weather_dataset = CustomDataset(val_weather['video_path'].values, val_weather['weather'].values)
val_weather_loader = DataLoader(val_weather_dataset, batch_size = CFG.batch_size, shuffle=False, num_workers=2)

In [ ]:
train_timing_dataset = CustomDataset(train_timing['video_path'].values, train_timing['timing'].values)
train_timing_loader = DataLoader(train_timing_dataset, batch_size = CFG.batch_size, shuffle=True, num_workers=2)

val_timing_dataset = CustomDataset(val_timing['video_path'].values, val_timing['timing'].values)
val_timing_loader = DataLoader(val_timing_dataset, batch_size = CFG.batch_size, shuffle=False, num_workers=2)

In [ ]:
train_loaders=[train_crush_loader,train_ego_loader,train_weather_loader,train_timing_loader]
val_loaders=[val_crush_loader,val_ego_loader,val_weather_loader,val_timing_loader]

In [ ]:
train_crush_dataset[0][0]

In [ ]:
#for tr_loader,val_loader in zip(train_loaders, val_loaders):
#    print(next(iter(tr_loader))[1], next(iter(val_loader))[1])

In [ ]:
#train_dataset[0][0].shape
#print(next(iter(train_crush_loader))[0].shape)
#print(next(iter(train_ego_loader))[0].shape)
#print(next(iter(train_weather_loader))[0].shape)
#print(next(iter(train_timing_loader))[0].shape)

## Model Define
- 동일 모델로 순서대로 4개 사용

In [ ]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=CFG.n_classes, fc_type='shallow', binary=True):
        super(BaseModel, self).__init__()
        self.fc_type=fc_type
        self.num_classes=num_classes
        
        #get backbone
        #self.backbone = r2plus1d_18(pretrained=True)
        self.backbone=getattr(torchvision.models.video,CFG.model_name)(pretrained=True)
        self.backbone.fc=self.get_fc()
        self.binary=binary
        
    def get_fc(self):
        if self.fc_type == 'deep':
            fc = nn.Sequential(nn.Linear(self.backbone.fc.in_features, self.backbone.fc.in_features//2),
                                        nn.BatchNorm1d(self.backbone.fc.in_features//2,  momentum=0.1),
                                        nn.ReLU(),
                                        nn.Linear(self.backbone.fc.in_features//2, self.num_classes)
                                        )
    
        elif self.fc_type == 'shallow':
            fc = nn.Linear(self.backbone.fc.in_features, self.num_classes)
        else:
            raise ValueError(f"Wrong fc-type input {self.fc_type}")
        return fc
    
    def forward(self, x):
        x = self.backbone(x)
            
        return x

### check model in-out

In [ ]:
#from torchsummary import summary
#model=BaseModel(fc_type='deep')
#model=model.to(device)
#summary(model, (3,50,720,1280))

## Train

In [ ]:
def train(model, criterion, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    
    if torch.cuda.device_count() > 1:
        print("Let's use", torch.cuda.device_count(), "GPUs!")
        model = nn.DataParallel(model)
    
    #criterion = nn.CrossEntropyLoss().to(device)
    criterion=criterion().to(device)
    best_val_score = 0
    best_model = None
    cnt=0
    for epoch in range(1, CFG.epochs+1):
        model.train()
        train_loss = []
        for videos, labels in tqdm(iter(train_loader)):
            videos = videos.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(videos)
            
            #print("target: ", labels)
            #print("output: ", output)
            #if model.module.binary==True: #because of DataParallel 
            #    labels=labels.float()
                
            #loss = criterion(output, labels.reshape(-1,1))
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val F1 : [{_val_score:.5f}]')
        
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_val_score < _val_score:
            best_val_score = _val_score
            best_model = model
            
            cnt=0
        else:
            print("early stopping count : {}".format(cnt))
            cnt+=1
        
        if cnt==CFG.earlystop:
            print("early stopping done")
            break
            
    return best_model

In [ ]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, trues = [], []
    
    with torch.no_grad():
        for videos, labels in tqdm(iter(val_loader)):
            videos = videos.to(device)
            labels = labels.to(device)
            
            #if model.module.binary==True: #because of DataParallel 
            #    labels=labels.float()
                
            logit = model(videos)
            
            #loss = criterion(logit, labels.reshape(-1,1))
            loss = criterion(logit, labels)
            val_loss.append(loss.item())
            
            preds += logit.argmax(1).detach().cpu().numpy().tolist() 

            trues += labels.detach().cpu().numpy().tolist()
        
        _val_loss = np.mean(val_loss)
    
    _val_score = f1_score(trues, preds, average='macro')
    return _val_loss, _val_score

## Run!!

In [ ]:
model_crush = BaseModel(num_classes=2, fc_type='shallow')
model_ego = BaseModel(num_classes=2, fc_type='shallow')
model_weather = BaseModel(num_classes=3, fc_type='shallow', binary=False)
model_timing = BaseModel(num_classes=2, fc_type='shallow')
models=[model_crush, model_ego, model_weather, model_timing]
name=["crush", "ego", "weather", "timing"]
#i=0
for i in range(4):
    model=models[i]
    model.eval()
    optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG.lr)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2, eta_min=0.00001)
    if i == 2: #weather
        criterion=nn.CrossEntropyLoss
        print("mutli-classification")
    else:
        criterion=nn.CrossEntropyLoss #여기 튜닝 고민 해봐야 할듯
        print("binary-classification")
    print("{}th model run".format(i))
    print("_"*100)
    infer_model = train(model, criterion, optimizer, train_loaders[i], val_loaders[i], scheduler, device)
    torch.save(infer_model.state_dict(), './weights/r3d_per_label_{}.pt'.format(name[i]))

    del infer_model

## Inference

In [ ]:
test = pd.read_csv('./test.csv')

In [ ]:
test_dataset = CustomDataset(test['video_path'].values, None)
test_loader = DataLoader(test_dataset, batch_size = CFG.batch_size, shuffle=False, num_workers=0)

In [ ]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    preds = []
    with torch.no_grad():
        for videos in tqdm(iter(test_loader)):
            videos = videos.to(device)
            
            logit = model(videos)

            preds += logit.argmax(1).detach().cpu().numpy().tolist()
    return preds

In [ ]:
i=0
from collections import OrderedDict

model_crush = BaseModel(num_classes=2, fc_type='shallow')
model_ego = BaseModel(num_classes=2, fc_type='shallow')
model_weather = BaseModel(num_classes=3, fc_type='shallow')
model_timing = BaseModel(num_classes=2, fc_type='shallow')
models=[model_crush, model_ego, model_weather, model_timing]
pred_tot=[]

for model in models:
    path='./weights/r3d_per_label_{}.pt'.format(name[i])
    state_dict=torch.load(path)
    keys = state_dict.keys()

    values = state_dict.values()
    
    new_keys = []
    
    for key in keys:
        new_key = key[7:]    # remove the 'module.'
        new_keys.append(new_key)
    
    new_dict = OrderedDict(list(zip(new_keys, values)))
    
    model.load_state_dict(new_dict)
    preds = inference(model, test_loader, device)
    pred_tot.append(preds)
    i+=1
    del model

## Submission

In [ ]:
submit = pd.read_csv('./sample_submission.csv')

In [ ]:
submit['crush'] = pred_tot[0]
submit['ego'] = pred_tot[1]
submit['weather'] = pred_tot[2]
submit['timing'] = pred_tot[3]
submit['label']=-1
submit.head()

In [ ]:
submit["crush"].value_counts(dropna=False).sort_index()

In [ ]:
submit["ego"].value_counts(dropna=False).sort_index()

In [ ]:
submit["weather"].value_counts(dropna=False).sort_index()

In [ ]:
submit["timing"].value_counts(dropna=False).sort_index()

In [ ]:
#label ensemble
def label_ensemble(submit):
    for i, row in submit.iterrows():
        #print(i)
        if row['crush']==0: # 0
            submit['label'][i]=0 
        else:
            if row['ego']==1: # 1~6
                if row['weather']==0: #1,2
                    if row['timing']==0:
                        submit['label'][i]=1
                    else:
                        submit['label'][i]=2
                        
                elif row['weather']==1:# 3,4
                    if row['timing']==0:
                        submit['label'][i]=3
                    else:
                        submit['label'][i]=4
                else:
                    if row['timing']==0:# 5,6
                        submit['label'][i]=5
                    else:
                        submit['label'][i]=6
    
            else: # 7~12
                if row['weather']==0: #7,8
                    if row['timing']==0:
                        submit['label'][i]=7
                    else:
                        submit['label'][i]=8
                        
                elif row['weather']==1:# 9,10
                    if row['timing']==0:
                        submit['label'][i]=9
                    else:
                        submit['label'][i]=10
                else:
                    if row['timing']==0:# 11,12
                        submit['label'][i]=11
                    else:
                        submit['label'][i]=12
        if submit['label'][i]==-1:
            print(row['crush'], row['ego'], row['weather'], row['timing'])
    #라벨 추가했으니 필요없는 열 이제 삭제
    submit = submit.drop(['crush'],axis=1)
    submit = submit.drop(['ego'],axis=1)
    submit = submit.drop(['weather'],axis=1)
    submit = submit.drop(['timing'],axis=1)
    
    return submit

submit=label_ensemble(submit)

In [ ]:
submit.to_csv('./r3d_labelEnsemble_submit.csv', index=False)

In [ ]:
submit["label"].value_counts(dropna=False).sort_index()

### test

In [ ]:
#https://dacon.io/en/competitions/official/236064/codeshare/7572?page=1&dtype=recent#
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.label_list = label_list
        self.transforms = transforms
        self.img_path_list = img_path_list
        
    def __getitem__(self, index):        
        images = self.get_frames(self.img_path_list[index])
                        
        if self.transforms is not None:
            res = self.transforms(**images)
            #print(images)
            images = torch.zeros((len(images), 3, CFG.img_size, CFG.img_size))
            #print(images.shape)
            images[0, :, :, :] = torch.Tensor(res["image"])
            for i in range(1, len(images)):
                images[i, :, :, :] = res[f"image{i}"]
        
        images=images.permute(1,0,2,3)
        
        if self.label_list is not None:
            label = self.label_list[index]
        
            return images, label
        else:
            return images

    def __len__(self):
        return len(self.img_path_list) 
    
    def get_frames(self, path):
        cap = cv2.VideoCapture(path)
        frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        imgs = []        
        for fidx in range(frames):
            _, img = cap.read()            
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            imgs.append(img)
        
        ret = {f"image{i}":imgs[i] for i in range(1, len(imgs))}
        ret['image'] = imgs[0]

        return ret
    
weather_transforms = A.Compose([
    A.CenterCrop(480,854,p=1.0),
    A.Resize(height=128, width=128),
    A.Superpixels(p=0.3),
    A.HorizontalFlip(0.5),
    ToTensorV2()
], p=1, additional_targets={f"image{i}":"image" for i in range(1, 50)})

other_transforms = A.Compose([
    A.CenterCrop(480,854,p=1.0),
    A.Resize(height=128, width=128),
    A.HorizontalFlip(0.5),
    ToTensorV2()
], p=1, additional_targets={f"image{i}":"image" for i in range(1, 50)})

test_transforms= A.Compose([
    A.CenterCrop(480,854,p=1.0),
    A.Resize(height=128, width=128),
    A.HorizontalFlip(0.5),
    ToTensorV2()
], p=1, additional_targets={f"image{i}":"image" for i in range(1, 50)})


train_crush_dataset = CustomDataset(train_crush['video_path'].values, train_crush['crush'].values,transforms=other_transforms)
train_crush_loader = DataLoader(train_crush_dataset, batch_size = CFG.batch_size, shuffle=True, num_workers=2)

val_crush_dataset = CustomDataset(val_crush['video_path'].values, val_crush['crush'].values,transforms=test_transforms)
val_crush_loader = DataLoader(val_crush_dataset, batch_size = CFG.batch_size, shuffle=False, num_workers=2)



In [ ]:
np.array(train_crush_dataset[0][0])